# 聚合函数
- 所有聚合函数都是放在django.db.models下面
- 聚合函数不能单独执行，要在某些方法中才能用，比如aggregate方法
- 聚合函数执行完毕后，系统会自动给这个聚合函数的返回值取名字，默认是  字段名__聚合函数名  。如果想自己命名，就在聚合函数前面加上自己的名字



## aggregate和annotate方法的区别
- 1.aggregate：返回使用聚合函数后的字段和值,是一个字典
- 2.annotate；在原来模型字段的基础之上添加一个使用了聚合函数的字段，并且在使用聚合函数的时候，会使用当前这个模型的主键进行分组(group by)。返回的是一个queryset
- annote原生语句：'SELECT xxx AVG(xxx) AS `avg` FROM xxx LEFT OUTER JOIN xxx ON xxx GROUP BY xxx ORDER BY NULL LIMIT 21'
- 示例：
        result = Tags.objects.annotate(avg=Avg("magazines__id")) #Tags为主表，magazines为子表名小写
        print(result)
        print(connection.queries)
        for r in result:
            print("{},{}".format(r.tag_name, r.avg))

## Avg()
- 导入：from django.db.models import Avg
- Avg("字段名")
- 使用：
        result = Magazines.objects.aggregate(Avg("id"))
        result = Magazines.objexts.aggregate(myavg=Avg("id")) #myavg是自己的命名
        print(result)  #{'id__avg': 31.5},id__avg是系统自动生成的，可以自己命名
        print(connection.queries)
        
## Count()
- 获取指定对象的个数,count可以传递参数 distinct=True,用来剔除那些重复的值，只保留一个
- result = Magazines.objects.aggregate(my_count=Count("id"))

## Max()和Min()
- 获取指定对象的最大值和最小值
- result = Magazines.objects.aggregate(my_max=Max("id"), my_min=Min("id"))

## Sum()
- 获取指定对象的和
- result = Magazines.objects.filter(release_time__year=2018).aggreate(total=sum("price"))
- 对于aggregate和annotate来说，只要返回的是querySet对象，就可以进行链式调用

# F表达式和Q表达式
- 动态的获取某个字段上的值，并且这个表达式，不会真正的去数据库中查询数据，相当于是起一个标识的作用
- F表达式是用来优化ORM操作数据库的，比如要给每位员工的工资加1000，按照正常流程来说，先把数据从数据库中取出，然后给每个数据的salary属性加上1000。这个过程就会先把数据提到python内存中，然后在python内存中做完运算，之后再保存到数据库中。而F表达式可以优化这个流程，它不需要先把数据从数据库中提取出来，他可以直接执行SQL语句，就将员工的工资增加1000
- F表达式不会马上从数据库中获取数据，而是在生成SQL语句时，动态的获取传给F表达式的值。
- 示例：
        result = Magazines.objects.update(title=F("id") + 10) #注意：F("id")是个数字的话只能和数字相加。若要改变title的内容，直接重新赋值即可，不用F
        result = Magazines.objects.filter(username=F('password')) #寻找username值和password值相等的数据
        

# Q表达式
- 使用Q表达式包裹查询条件，可以在条件之间进行多种操作，与或非(&|~)等，从而实现一些复杂的查询操作
- result = Book.objects.filter(Q(price__gte=100)|~Q(pin_fen__gte=8)) #选取价格大于等于100或者评分在8以下的

# 聚合函数和querySet API的区别
- 聚合函数是用在括号里面，querySetAPI是用在括号外面。聚合函数是对数据进行加减乘除，querySetAPI是对querySet对象进行一系列的操作